# 四轴示教

In [1]:
import os
import sys
# 添加pcan_cybergear库的路径
sys.path.append(os.path.join("..", "cybergear"))

from pcan_cybergear import CANMotorController
import can
import logging
import time
# Initialize logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [2]:
# Connect to the CAN bus with 1 Mbit/s bitrate
bus = can.interface.Bus(interface="pcan", channel="PCAN_USBBUS1", bitrate=1000000)
motor1 = CANMotorController(bus, motor_id=101, main_can_id=254)
motor2 = CANMotorController(bus, motor_id=102, main_can_id=254)
motor3 = CANMotorController(bus, motor_id=103, main_can_id=254)
motor4 = CANMotorController(bus, motor_id=104, main_can_id=254)

motors = [motor1, motor2, motor3, motor4]

In [ ]:
# 写参数表
for motor in motors:
    motor.write_param_table("limit_cur", 1.5)
    motor.write_param_table("loc_kp", 8)
    motor.write_param_table("spd_kp", 2)
    motor.write_param_table("spd_ki", 0.03)
motor2.write_param_table("limit_cur", 3)

## 初始化

In [ ]:
for motor in motors:
    motor.disable()
    motor.set_0_pos()
    motor.set_run_mode(motor.RunModes.POSITION_MODE) # 位置模式
    motor.write_single_param("loc_ref", value=0) # 目标位置

motor1.write_single_param("limit_spd", value=1) # 最大速度 rad/s
motor2.write_single_param("limit_spd", value=1) # 最大速度 rad/s
motor3.write_single_param("limit_spd", value=1) # 最大速度 rad/s
motor4.write_single_param("limit_spd", value=1) # 最大速度 rad/s

In [ ]:
for motor in motors:
    motor.enable()

In [ ]:
import time
motor4.write_single_param("loc_ref", value=-0.5)
time.sleep(1.5)
motor4.write_single_param("loc_ref", value=0.5)
time.sleep(1.5)
motor4.write_single_param("loc_ref", value=0)

In [ ]:
for motor in motors:
    motor.disable()
    motor.set_0_pos()

# 示教

In [8]:
import csv
import time

# 保存点位信息到CSV文件
def save_positions_to_csv(positions, filename="positions.csv"):
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["motor1", "motor2", "motor3", "motor4"])
        for position in positions:
            writer.writerow([position[0], position[1], position[2], position[3]])

# 从CSV文件加载点位信息
def load_positions_from_csv(filename="positions.csv"):
    positions = []
    with open(filename, mode='r') as file:
        csv_reader = csv.reader(file)
        next(csv_reader)  # skip header
        for row in csv_reader:
            positions.append([float(row[0]), float(row[1]), float(row[2]), float(row[3])])
    return positions


In [ ]:
positions = []

while True:
    action = input("Press Enter to record the current position, or 'x' to exit: ")

    if action == "x":
        break

    # 使用星号解包确保兼容性
    _, pos1, *_ = motor1.write_single_param("loc_ref", value=0)
    _, pos2, *_ = motor2.write_single_param("loc_ref", value=0)
    _, pos3, *_ = motor3.write_single_param("loc_ref", value=0)
    _, pos4, *_ = motor4.write_single_param("loc_ref", value=0)
    
    pos = [pos1, pos2, pos3, pos4]
    logging.info(pos)
    positions.append(pos)

In [ ]:
len(positions)

In [11]:
save_positions_to_csv(positions)

In [12]:
import pandas as pd
pd.read_csv("positions.csv")

,motor1,motor2,motor3,motor4
0,0.000191,0.084497,0.995079,0.000954
1,0.642977,0.151637,1.585222,0.000954
2,0.691043,0.336271,2.358091,0.000954
3,0.260739,-0.918021,1.823262,0.000954
4,0.219539,-1.151102,1.567292,0.000954
5,0.201228,-0.296216,1.575685,0.000954
6,-0.539979,-0.144770,1.495575,0.000572
7,-0.290494,-0.086404,0.415236,0.000954


In [ ]:
motor1.write_single_param("limit_spd", value=1) # 最大速度 rad/s
motor2.write_single_param("limit_spd", value=1) # 最大速度 rad/s
motor3.write_single_param("limit_spd", value=1) # 最大速度 rad/s
motor4.write_single_param("limit_spd", value=1) # 最大速度 rad/s

In [ ]:
for motor in motors:
    motor.write_single_param("loc_ref", value=0)
    motor.enable()

In [ ]:
loaded_positions = load_positions_from_csv()

for position in loaded_positions:
    logging.info(position)
    for i, motor in enumerate(motors):
        motor.write_single_param("loc_ref", value=position[i])
    time.sleep(1.5)  # 等待电机移动到目标位置

for motor in motors:
    motor.write_single_param("loc_ref", value=0)

In [ ]:
for motor in motors:
    motor.disable()